In [9]:
import importlib
import scraper

importlib.reload(scraper)  # Forces reloading of the scraper module

from scraper import fetch_sold_listings, get_listing_details, debug_pagination, clean_price

In [5]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# options = webdriver.ChromeOptions()

# # Automatically match ChromeDriver with installed Chrome version
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# driver.get("https://google.com")  # Test if Chrome opens correctly

# print("Chrome opened successfully!")
# driver.quit()

In [6]:
prices_df = get_listing_details("housesigma_page_1.html")

print(prices_df['Address'][0])
print(prices_df)

Reading html file
623 - 88 Colgate Ave , Toronto - South Riverdale
   Asking Price  Sold Price  \
0      $998,000    $975,000   
1    $2,490,000  $2,498,000   
2      $589,900    $575,000   
3      $669,990    $620,000   
4      $945,000    $920,000   
5      $899,000    $930,000   
6    $3,399,000  $3,215,000   
7      $599,900    $570,000   
8      $649,000    $625,000   
9    $1,149,900  $1,150,000   
10     $440,000    $425,000   
11   $1,188,000  $1,300,000   
12     $685,000    $680,000   
13     $999,786  $1,125,000   
14   $1,680,000  $1,680,000   
15     $299,999    $310,500   
16     $749,900    $745,000   
17     $619,990    $592,500   
18   $5,355,000  $4,412,000   
19     $794,900    $775,000   

                                              Address           Unit Type  \
0    623 - 88 Colgate Ave , Toronto - South Riverdale           Condo Apt   
1       19 Dalewood Dr , Richmond Hill - Bayview Hill            Detached   
2   3215 - 21 Iceboat Terr , Toronto - Waterfront 

In [7]:
# Run the scraper
url = "https://housesigma.com/on/sold/map/?status=sold&lat=43.715564&lon=-79.418602&zoom=10&page=1&view=list"

# debug_pagination(url)

sold_data_df = fetch_sold_listings(url, 40)

Opening HouseSigma...
Scraping: https://housesigma.com/on/sold/map/?status=sold&lat=43.715564&lon=-79.418602&zoom=10&page=1&view=list
Reloaded the url, waiting for the page to load
✅ Saved HTML: housesigma_page_1.html

Retrieving data from the page
Reading html file
  Asking Price  Sold Price                                            Address  \
0     $998,000    $975,000   623 - 88 Colgate Ave , Toronto - South Riverdale   
1   $2,490,000  $2,498,000      19 Dalewood Dr , Richmond Hill - Bayview Hill   
2     $589,900    $575,000  3215 - 21 Iceboat Terr , Toronto - Waterfront ...   
3   $1,249,000  $1,250,000                   54 William St , Toronto - Weston   
4     $945,000    $920,000     406 - 21 Grand Magazine St , Toronto - Niagara   

   Unit Type Bedrooms Bathrooms Postal Code  
0  Condo Apt        2         2      M4M0A6  
1   Detached      4+2         6      L4B3C3  
2  Condo Apt      1+1         1      M5V4A9  
3   Detached      3+1         2      M9N2G7  
4  Condo Apt    

UnboundLocalError: cannot access local variable 'bathroom_count' where it is not associated with a value

In [1]:
print("Dataframe shape: ", sold_data_df.shape)

print(sold_data_df.head(20))

NameError: name 'sold_data_df' is not defined

In [72]:
print(sold_data_df.columns)

# Apply the cleaning function to both columns
price_columns = ["Asking Price", "Sold Price"]
sold_data_df[price_columns] = sold_data_df[price_columns].applymap(clean_price)

print(sold_data_df.head(20))

Index(['Asking Price', 'Sold Price', 'Address', 'Unit Type', 'Bedrooms',
       'Bathrooms', 'Postal Code'],
      dtype='object')
    Asking Price  Sold Price  \
0       998000.0    975000.0   
1      2490000.0   2498000.0   
2       589900.0    575000.0   
3       669990.0    620000.0   
4       945000.0    920000.0   
5       899000.0    930000.0   
6      3399000.0   3215000.0   
7       599900.0    570000.0   
8       649000.0    625000.0   
9      1149900.0   1150000.0   
10      440000.0    425000.0   
11     1188000.0   1300000.0   
12      685000.0    680000.0   
13      999786.0   1125000.0   
14     1680000.0   1680000.0   
15      299999.0    310500.0   
16      749900.0    745000.0   
17      619990.0    592500.0   
18     5355000.0   4412000.0   
19      794900.0    775000.0   

                                              Address           Unit Type  \
0    623 - 88 Colgate Ave , Toronto - South Riverdale           Condo Apt   
1       19 Dalewood Dr , Richmond Hill - B

C:\Users\USER\AppData\Local\Temp\ipykernel_1604\2337118698.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sold_data_df[price_columns] = sold_data_df[price_columns].applymap(clean_price)


In [74]:
print(sold_data_df.columns)

# Average asking and sold prices by unit type
avg_prices = (
    sold_data_df.groupby("Unit Type")
    .agg(
        Asking_Price_Avg=("Asking Price", "mean"),
        Sold_Price_Avg=("Sold Price", "mean"),
        Count=("Asking Price", "count")  # Counting the number of observations
    )
    .reset_index()
)

# Format the average prices and counts for better readability
avg_prices["Asking_Price_Avg"] = avg_prices["Asking_Price_Avg"].apply(lambda x: f"${x:,.0f}")
avg_prices["Sold_Price_Avg"] = avg_prices["Sold_Price_Avg"].apply(lambda x: f"${x:,.0f}")
avg_prices["Count"] = avg_prices["Count"].apply(lambda x: f"{x:,}")

# Display the formatted table
print("\nAverage Asking and Sold Price by Unit Type:\n")
print(avg_prices.to_string(index=False))

# Group by Unit Type and Bedrooms, then calculate average Asking and Sold Prices
avg_prices_with_count = (
    sold_data_df.groupby(["Unit Type", "Bedrooms"])
    .agg(
        Count=("Asking Price", "count") ,
        Asking_Price_Avg=("Asking Price", "mean"),
        Sold_Price_Avg=("Sold Price", "mean"),
    )
    .reset_index()
)
# Print the results
print("\nAverage Asking and Sold Price by Unit Type and Number of Bedrooms:")
print(avg_prices_with_count)


Index(['Asking Price', 'Sold Price', 'Address', 'Unit Type', 'Bedrooms',
       'Bathrooms', 'Postal Code'],
      dtype='object')

Average Asking and Sold Price by Unit Type:

                     Unit Type Asking_Price_Avg Sold_Price_Avg Count
Apartment Unit, Condo/Apt Unit         $729,000       $708,000     2
                     Co-Op Apt         $411,833       $400,083     6
                     Condo Apt         $755,495       $741,065   265
               Condo Townhouse       $1,003,326       $990,095    48
                      Detached       $1,706,622     $1,696,627   177
                        Duplex       $2,165,000     $1,945,000     2
            Freehold Townhouse       $1,166,023     $1,209,716    25
                          Link       $1,195,431     $1,260,311     9
                         Other         $868,900       $840,000     1
                 Parking Space          $30,000        $26,000     1
                 Semi-Detached       $1,090,730     $1,160,235  